## Plot gene set comparison

In this notebook, we want to compare prediction of mutations in the genes from our cancer gene set, derived from [Vogelstein et al. 2013](https://science.sciencemag.org/content/339/6127/1546), with two other sets of potentially relevant genes. These are:

* The most frequently mutated genes in TCGA
* A set of random genes in TCGA, that meet our mutation count threshold for 2 or more cancer types
 
We selected enough genes in each of these gene sets to (approximately) match the count of the Vogelstein et al. gene set.
 
In these experiments we only used gene expression data, and we used the set of TCGA samples that have both gene expression and MC3 somatic mutation data. The files analyzed in this notebook were generated by the `run_mutation_prediction.py` script.

Notebook parameters:
* SIG_ALPHA (float): significance cutoff (after FDR correction)

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text

import mpmp.config as cfg
import mpmp.utilities.analysis_utilities as au
import mpmp.utilities.plot_utilities as plu

In [2]:
# set results directory
vogelstein_results_dir = Path(cfg.results_dirs['mutation'],
                              'vogelstein_expression_only',
                              'gene').resolve()
top_50_results_dir = Path(cfg.results_dirs['mutation'],
                          'top_50_expression_only',
                          'gene').resolve()
random_50_results_dir = Path(cfg.results_dirs['mutation'],
                             '50_random_expression_only',
                             'gene').resolve()

# set significance cutoff after FDR correction
SIG_ALPHA = 0.001

# if True, save figures to ./images directory
SAVE_FIGS = True

In [3]:
# load raw data
vogelstein_df = au.load_stratified_prediction_results(vogelstein_results_dir, 'gene')
vogelstein_df = vogelstein_df[vogelstein_df.training_data.isin(['expression'])]
vogelstein_df['gene_set'] = 'vogelstein'

# make sure that we're correctly pointing to raw data for non-methylation data types
# and that we have data for two replicates (two random seeds)
print(vogelstein_df.shape)
print(vogelstein_df.seed.unique())
print(vogelstein_df.training_data.unique())
vogelstein_df.head()

(4080, 10)
[42  1]
['expression']


,auroc,aupr,identifier,training_data,signal,seed,data_type,fold,experiment,gene_set
0,0.99980,0.99808,MAP3K1,expression,signal,42,train,0,gene,vogelstein
1,0.75084,0.43962,MAP3K1,expression,signal,42,test,0,gene,vogelstein
2,0.68143,0.30787,MAP3K1,expression,signal,42,cv,0,gene,vogelstein
3,0.99999,0.99989,MAP3K1,expression,signal,42,train,1,gene,vogelstein
4,0.66563,0.43830,MAP3K1,expression,signal,42,test,1,gene,vogelstein


In [4]:
# load raw data
top_50_df = au.load_stratified_prediction_results(top_50_results_dir, 'gene')
top_50_df = top_50_df[top_50_df.training_data.isin(['expression'])]
top_50_df['gene_set'] = 'top_50'

# make sure that we're correctly pointing to raw data for non-methylation data types
# and that we have data for two replicates (two random seeds)
print(top_50_df.shape)
print(top_50_df.seed.unique())
print(top_50_df.training_data.unique())
top_50_df.head()

(4080, 10)
[42  1]
['expression']


,auroc,aupr,identifier,training_data,signal,seed,data_type,fold,experiment,gene_set
0,0.98433,0.98653,TP53,expression,shuffled,42,train,0,gene,top_50
1,0.50011,0.48180,TP53,expression,shuffled,42,test,0,gene,top_50
2,0.52088,0.49482,TP53,expression,shuffled,42,cv,0,gene,top_50
3,0.97987,0.97430,TP53,expression,shuffled,42,train,1,gene,top_50
4,0.50415,0.49414,TP53,expression,shuffled,42,test,1,gene,top_50


In [5]:
# load raw data
random_50_df = au.load_stratified_prediction_results(random_50_results_dir, 'gene')
random_50_df = random_50_df[random_50_df.training_data.isin(['expression'])]
random_50_df['gene_set'] = 'random_50'

# make sure that we're correctly pointing to raw data for non-methylation data types
# and that we have data for two replicates (two random seeds)
print(random_50_df.shape)
print(random_50_df.seed.unique())
print(random_50_df.training_data.unique())
random_50_df.head()

(4080, 10)
[42  1]
['expression']


,auroc,aupr,identifier,training_data,signal,seed,data_type,fold,experiment,gene_set
0,0.98433,0.98653,TP53,expression,shuffled,42,train,0,gene,random_50
1,0.50011,0.48180,TP53,expression,shuffled,42,test,0,gene,random_50
2,0.52088,0.49482,TP53,expression,shuffled,42,cv,0,gene,random_50
3,0.97987,0.97430,TP53,expression,shuffled,42,train,1,gene,random_50
4,0.50415,0.49414,TP53,expression,shuffled,42,test,1,gene,random_50


In [6]:
# combine results dataframes
results_df = (
    pd.concat((vogelstein_df, top_50_df, random_50_df))
      .drop(columns=['training_data', 'experiment'])
      .rename(columns={'gene_set': 'training_data'})
)

all_results_df = au.compare_all_data_types(results_df,
                                           SIG_ALPHA,
                                           filter_genes=False)

all_results_df['nlog10_p'] = -np.log10(all_results_df.corr_pval)
all_results_df.sort_values(by='p_value').head(10)

,gene,delta_mean,p_value,corr_pval,reject_null,training_data,nlog10_p
82,TP53,0.418908,3.030301e-13,2.575756e-11,True,vogelstein,10.589095
77,TP53,0.418908,3.030301e-13,2.575756e-11,True,top_50,10.589095
77,TP53,0.418908,3.030301e-13,2.575756e-11,True,random_50,10.589095
23,EGFR,0.603976,6.247747e-11,2.655293e-09,True,vogelstein,8.575888
17,CDKN2A,0.387666,1.552123e-10,4.192733e-09,True,vogelstein,8.377503
2,APC,0.593855,1.973051e-10,4.192733e-09,True,vogelstein,8.377503
5,APC,0.593855,1.973051e-10,8.385466e-09,True,top_50,8.076473
69,PTEN,0.576172,1.579724e-09,2.685531e-08,True,vogelstein,7.570970
64,PTEN,0.576172,1.579724e-09,4.121261e-08,True,top_50,7.384970
51,KRAS,0.468161,1.939417e-09,2.747507e-08,True,vogelstein,7.561061


In [ ]:
sns.set({'figure.figsize': (24, 6)})
sns.set_style('whitegrid')
fig, axarr = plt.subplots(1, 3)

gene_set_map = {
    'random_50': 'random',
    'top_50': 'most mutated',
    'vogelstein': 'Vogelstein et al.'
}
all_results_df.training_data.replace(to_replace=gene_set_map, inplace=True)

plu.plot_volcano_baseline(all_results_df,
                          axarr,
                          gene_set_map,
                          SIG_ALPHA,
                          verbose=True,
                          color_overlap=True)

if SAVE_FIGS:
    images_dir = Path(cfg.images_dirs['mutation'])
    images_dir.mkdir(exist_ok=True)
    plt.savefig(images_dir / 'expression_vs_shuffled.svg', bbox_inches='tight')
    plt.savefig(images_dir / 'expression_vs_shuffled.png',
                dpi=300, bbox_inches='tight')

random: 12/85
most mutated: 15/85


In [ ]:
sns.set({'figure.figsize': (8, 6)})
sns.set_style('whitegrid')
fig, axarr = plt.subplots(1, 1)

gene_set_map = {
    'vogelstein': 'Vogelstein et al.'
}
all_results_df.training_data.replace(to_replace=gene_set_map, inplace=True)

plu.plot_volcano_baseline(all_results_df,
                          axarr,
                          gene_set_map,
                          SIG_ALPHA,
                          verbose=True)

if SAVE_FIGS:
    images_dir = Path(cfg.images_dirs['mutation'])
    images_dir.mkdir(exist_ok=True)
    plt.savefig(images_dir / 'expression_vogelstein.svg', bbox_inches='tight')
    plt.savefig(images_dir / 'expression_vogelstein.png',
                dpi=300, bbox_inches='tight')

In [ ]:
sns.set({'figure.figsize': (10, 6)})
sns.set_style('whitegrid')
# we want these colors to be different than the expression/methylation ones
sns.set_palette('Set2')
fig, axarr = plt.subplots(1, 1)

all_results_df = (all_results_df
   .rename(columns={'training_data': 'gene_set'})
   .sort_values(by='gene_set', ascending=False)
)

# plot mean performance over all genes in Vogelstein dataset
ax = axarr
sns.boxplot(data=all_results_df, x='gene_set', y='delta_mean',
            notch=True, ax=ax)
ax.set_title('Performance distribution for all genes in each gene set, gene expression only', size=14)
ax.set_xlabel('Target gene set', size=13)
ax.set_ylabel('AUPR(signal) - AUPR(shuffled)', size=13)
ax.set_ylim(-0.2, max(all_results_df.delta_mean + 0.05))
for tick in ax.get_xticklabels():
    tick.set_fontsize(13)
    
plt.tight_layout()

if SAVE_FIGS:
    plt.savefig(images_dir / 'expression_boxes.svg', bbox_inches='tight')
    plt.savefig(images_dir / 'expression_boxes.png',
                dpi=300, bbox_inches='tight')

In [ ]:
sns.set({'figure.figsize': (18, 6)})
sns.set_style('whitegrid')
# we want these colors to be different than the expression/methylation ones
sns.set_palette('Set2')
fig, axarr = plt.subplots(1, 2)

all_results_df.replace({'gene_set': gene_set_map}, inplace=True)

# plot mean performance over all genes in Vogelstein dataset
ax = axarr[0]
sns.stripplot(data=all_results_df, x='gene_set', y='delta_mean', ax=ax)
ax.set_title('Prediction for all genes, performance vs. gene set')
ax.set_xlabel('Target gene set')
ax.set_ylabel('AUPR(signal) - AUPR(shuffled)')
ax.set_ylim(-0.2, max(all_results_df.delta_mean + 0.05))

# plot mean performance for genes that are significant for at least one data type
ax = axarr[1]
gene_list = all_results_df[all_results_df.reject_null == True].gene.unique()
print(gene_list.shape)
print(gene_list)
sns.stripplot(data=all_results_df[all_results_df.gene.isin(gene_list)],
              x='gene_set', y='delta_mean', ax=ax)
ax.set_title('Prediction for significant genes, performance vs. gene set')
ax.set_xlabel('Target gene set')
ax.set_ylabel('AUPR(signal) - AUPR(shuffled)')
ax.set_ylim(-0.2, max(all_results_df.delta_mean + 0.05))

### Calculate gene set overlap
 
Of the significantly predictable genes in the top/random gene sets, how many of them are in the Vogelstein gene set?

In [ ]:
from venn import venn

# first look at overlap of all genes
genes_in_gene_set = {}
for gene_set in all_results_df.gene_set.unique():
    gene_list = all_results_df[all_results_df.gene_set == gene_set].gene.unique()
    print(gene_set, len(gene_list))
    genes_in_gene_set[gene_set] = set(gene_list)

venn(genes_in_gene_set)
plt.title('Gene overlap between all genes in gene set', size=14)

In [ ]:
# now look at overlap of significant genes
genes_in_gene_set = {}
for gene_set in all_results_df.gene_set.unique():
    gene_list = all_results_df[(all_results_df.gene_set == gene_set) &
                               (all_results_df.reject_null)].gene.unique()
    print(gene_set, len(gene_list))
    genes_in_gene_set[gene_set] = set(gene_list)

venn(genes_in_gene_set)
plt.title('Gene overlap between significantly predictable genes in gene set', size=14)